<a href="https://colab.research.google.com/github/weibb123/Unsupervised_sentiment_analysis_on_Youtube/blob/main/grab_youtube_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade google-api-python-client

     |████████████████████████████████| 7.7 MB 5.5 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.284 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 2.28.0 which is incompatible.


In [ ]:
import json
from csv import writer
from apiclient.discovery import build
import pandas as pd
import pickle
import urllib.request
import urllib

In [ ]:
#key = 'Your own youtube videoID here'        
# videoId = 'aircAruvnKk' # 3BLUE1BROWN video
ndvideoId = 'wW1lY5jFNcQ' # Biden and Trump video

In [ ]:
def build_service():
    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [ ]:
def get_comments(part='snippet', 
                 maxResults=100, 
                 textFormat='plainText',
                 order='time',
                 videoId= ndvideoId,
                 csv_filename="PresidentialDebate_comments"
                 ):

    #3 create empty lists to store desired information
    comments = []

    # build our service from path/to/apikey
    service = build_service()
    
    #4 make an API call using our service
    response = service.commentThreads().list(
        part=part,
        maxResults=maxResults,
        textFormat='plainText',
        order=order,
        videoId= ndvideoId
    ).execute()

    while response: # this loop will continue to run until you max out your quota

        for item in response['items']:
            #5 index item for desired data features
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            
            #6 append to lists
            comments.append(comment)
            
        try:
            if 'nextPageToken' in response:
                response = service.commentThreads().list(
                    part=part,
                    maxResults=maxResults,
                    textFormat=textFormat,
                    order=order,
                    videoId= ndvideoId,
                    pageToken=response['nextPageToken']
                ).execute()
            else:
                break
        except: break

    #9 return our data of interest
    return {
        'comment': comments
        
    }

In [ ]:
def get_vid_title(vidid):
    VideoID = videoId
    params = {"format": "json", "url": "https://www.youtube.com/watch?v=wW1lY5jFNcQ" % vidid}
    url = "https://www.youtube.com/watch?v=wW1lY5jFNcQ"
    query_string = urllib.parse.urlencode(params)
    url = url + "?" + query_string

    with urllib.request.urlopen(url) as response:
        response_text = response.read()
        data = json.loads(response_text.decode())
        # print(data['title'])
        return data['title']



In [ ]:
if __name__ == '__main__':
    comments = get_comments()
    df = pd.DataFrame(comments)
    print(df.shape)
    print(df.head())
    df.to_csv('./PresidentialDebate_comments.csv')

(52493, 1)
                                             comment
0  Joe Biden: "Show us your tax returns". What a ...
1   We see now who told us the truth in this debate.
2  Joe doesn't trust Trump, but he trusts Fauci! ...
3               Here's  the deal: Let's Go Brandon!!
4  Blue vs. Red; The 2020 Election | Marcus Ward ...


In [ ]:
 ## lOAD DATA to my drive...
 
from google.colab import drive
drive.mount('drive')

df.to_csv('./PresidentialDebate_comments.csv')
!cp PresidentialDebate_comments.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


https://towardsdatascience.com/how-to-build-your-own-dataset-of-youtube-comments-39a1e57aade